# Imports

In [1]:
import pandas as pd
import vectorbtpro as vbt
import numpy as np

# Initialization

In [2]:
vbt.settings.wrapping ["freq"]                = "1m"
vbt.settings.portfolio['init_cash']           = 10000

# Settings and Parameters

In [3]:
#pickle_files_path = "../data/75_prediction_normalized/*.pkl"
pickle_files_path = "../data/LSTM_pw38_lb1500_db90m_july202023/*.pkl"
pickle_files_path = "../data/LSTM_pw75_lb250_bt1000_db90m_july202023/*.pkl"
prediction_window = 75

excel_output_file_name  = f"../results/{pickle_files_path.split('/')[-2] + '.xlsx'}"

In [4]:
pickle_files_path.split("/")[-2]

'LSTM_pw75_lb250_bt1000_db90m_july202023'

# Local Imports

In [5]:
import os
import sys

sys.path.append(os.getcwd())

In [6]:
from lstm_analysis_utils import (read_pickle_files_into_df, add_forward_prices_to_df, generate_fwd_actual_column
                                 , generate_df_with_euclidean_distances, calculate_slopes, calculate_correlation_slopes
                                )
from lstm_results_utils import (export_results, store_backtest_results)
from lstm_analysis_constants import ActionType, LSTM_REVERSAL_EXITS_BACKTEST_RESULT_KEY, LSTM_PREDICTION_WINDOW_EXITS_BACKTEST_RESULT_KEY
from quantile_value import QuantileBand, QuantileValue, generate_quantile_bands

# Processing

In [7]:
df = read_pickle_files_into_df(pickle_files_path)

In [8]:
add_forward_prices_to_df(df, prediction_window)
df = df.copy()  # for large prediction_window size, the copy() call eliminates the fragmented dataframe warning

In [9]:
generate_fwd_actual_column(df)

In [10]:
df = generate_df_with_euclidean_distances(df, prediction_window)

In [11]:
calculate_slopes(df)

In [12]:
calculate_correlation_slopes(df)

Correlation between Euclidean distance between long array and short array and future actual results:  0.16328526361580675
Correlation between difference in long minus short predictions and future actual results for longs:  0.7453795735601428
Correlation between difference in long minus short predictions and future actual results for shorts:  0.7620750640854376
Correlation between long slopes and future results: 0.02332456091074073
Correlation between short slopes and future results: -0.004838081923707713


# Running backtest

- ### Storing the results of the backtests

In [13]:
results_as_list  = []

- ### Baseline backtest - just listen to LSTM

In [14]:
# Exits are from reversals, as our LSTM model doesn't produce exit signals
entries         = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_LONG  ), True, False))
exits           = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_LONG ), True, False))
short_entries   = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_SHORT ), True, False))
short_exits     = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_SHORT), True, False))

pf = vbt.Portfolio.from_signals(
    high                = df['BTCUSDT_High'],
    low                 = df['BTCUSDT_Low'],
    open                = df['BTCUSDT_Open'],
    close               = df['BTCUSDT_Close'],
    entries             = entries, # commented out for a short only backtest
    exits               = exits,
    short_entries       = short_entries,
    short_exits         = short_exits,    
    time_delta_format   = 'Rows', # Use the row index to calculate the time delta    
    )

store_backtest_results(LSTM_REVERSAL_EXITS_BACKTEST_RESULT_KEY, pf, results_as_list)

--------LSTM_only_reversal_exits--------
Total Return [%]: -18.23
Win Rate [%]: 49.89
Total Trades: 451


In [15]:
# Exits are controlled by td_stop
entries         = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_LONG  ), True, False))
short_entries   = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_SHORT ), True, False))

pf = vbt.Portfolio.from_signals(
    high                = df['BTCUSDT_High'],
    low                 = df['BTCUSDT_Low'],
    open                = df['BTCUSDT_Open'],
    close               = df['BTCUSDT_Close'],
    entries             = entries, # commented out for a short only backtest    
    short_entries       = short_entries,     
    td_stop             = prediction_window,
    time_delta_format   = 'Rows', # Use the row index to calculate the time delta    
    )
store_backtest_results(LSTM_PREDICTION_WINDOW_EXITS_BACKTEST_RESULT_KEY, pf, results_as_list)

--------LSTM_only_prediction_window_exits--------
Total Return [%]: -22.60
Win Rate [%]: 49.30
Total Trades: 718


- ### Using different slope quantiles

In [16]:
num_quantiles               = 5
threshold_increment         = 0.0001
quantiles                   = np.linspace(0, 1, num=num_quantiles + 1)

- #### long_slope and short_slope

In [17]:
long_slope_quantile_bands  = generate_quantile_bands(df["long_slope"].quantile(quantiles))
short_slope_quantile_bands = generate_quantile_bands(df["short_slope"].quantile(quantiles))  

In [18]:
for long_band in long_slope_quantile_bands:
  for short_band in short_slope_quantile_bands:    
    entry_slope_threshold       = [x for x in np.arange(long_band.lower_bound.value , long_band.upper_bound.value , threshold_increment)]
    short_entry_slope_threshold = [x for x in np.arange(short_band.lower_bound.value, short_band.upper_bound.value, threshold_increment)]

    for t1 in entry_slope_threshold:
      # TODO: do another test with exits vs. td_stop
      #exit_t1_threshold = t1 * 0.5
      for t2 in short_entry_slope_threshold:
        #exit_t2_threshold = t2 * 0.5
        entries       = pd.Series(np.where((df['long_slope' ] > t1  ), True, False))
        short_entries = pd.Series(np.where((df['short_slope'] < -t2 ), True, False))

        #exits = 
        #short_exits =

        num_trades = (entries == True).sum() + (short_entries == True).sum()

        if num_trades > 0:    
          pf = vbt.Portfolio.from_signals(
              high              = df['BTCUSDT_High'],
              low               = df['BTCUSDT_Low'],
              open              = df['BTCUSDT_Open'],
              close             = df['BTCUSDT_Close'],
              entries           = entries, # commented out for a short only backtest          
              short_entries     = short_entries,
              td_stop           = prediction_window, # Hold on to the position for 8 bars
              time_delta_format = 'Rows', # Use the row index to calculate the time delta              
              accumulate        = False,
              # sl_stop = 0.005,
              )    
          
          key = f"entry_slope_threshold_{t1}_short_entry_slope_threshold_{t2}"
          store_backtest_results(key, pf, results_as_list)      

--------entry_slope_threshold_-0.0010526288827384688_short_entry_slope_threshold_-0.001052630460031744--------
Total Return [%]: -2.16
Win Rate [%]: 0.00
Total Trades: 1
--------entry_slope_threshold_-0.0010526288827384688_short_entry_slope_threshold_-0.0009526304600317439--------
Total Return [%]: -31.28
Win Rate [%]: 47.70
Total Trades: 369
--------entry_slope_threshold_-0.0010526288827384688_short_entry_slope_threshold_-0.0008526304600317439--------
Total Return [%]: -49.59
Win Rate [%]: 49.88
Total Trades: 425
--------entry_slope_threshold_-0.0009526288827384688_short_entry_slope_threshold_-0.001052630460031744--------
Total Return [%]: 137.33
Win Rate [%]: 52.00
Total Trades: 375
--------entry_slope_threshold_-0.0009526288827384688_short_entry_slope_threshold_-0.0009526304600317439--------
Total Return [%]: 2.35
Win Rate [%]: 50.15
Total Trades: 1940
--------entry_slope_threshold_-0.0009526288827384688_short_entry_slope_threshold_-0.0008526304600317439--------
Total Return [%]: -1

# Exporting the results

In [19]:
results_df = export_results(results_as_list)
results_df.to_excel(excel_output_file_name)

- #### long_minus_short

In [20]:
pf.stats()



/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


Start                                  1641686580.00000
End                                    1668579480.00000
Period                                 28 days 15:15:00
Start Value                                 10000.00000
Min Value                                    8449.91412
Max Value                                   14179.15241
End Value                                   10103.21042
Total Return [%]                                1.03210
Benchmark Return [%]                          -59.50205
Total Time Exposure [%]                        60.59901
Max Gross Exposure [%]                        120.98842
Max Drawdown [%]                               40.40607
Max Drawdown Duration                  17 days 00:25:00
Total Orders                                        757
Total Fees Paid                                 0.00000
Total Trades                                        495
Win Rate [%]                                   52.92929
Best Trade [%]                                  

In [21]:
pf.trades.count()

495

In [22]:
short_slope_quantile_values

NameError: name 'short_slope_quantile_values' is not defined

In [ ]:
entry_slope_threshold       = [x for x in np.arange(-0.01754, -0.01206, 0.0001)]
short_entry_slope_threshold = [x for x in np.arange(-0.01754, -0.01123, 0.0001)]

In [ ]:
for i in range(len(long_slope_quantile_values)):
    label = long_slope_quantile_values.index[i]
    value = long_slope_quantile_values.iloc[i]

    print(f"label = {label}, value = {value}")

In [ ]:
# for label, entry in temp.items():
#     print(f"label = {label}, entry = {entry}")

In [ ]:
# for i in range(len(temp)):
#     label = temp.index[i]
#     value = temp.iloc[i]

#     print(f"label = {label}, value = {value}")

In [ ]:
pf.total_return

In [ ]:
pf.trades.win_rate

In [ ]:
temp = pf.trades.win_rate * 100
type(temp)

# Convert temp to a string
temp = temp.astype(str)
print(temp)

- ### Other backtests - work in progress and code may not execute at all!!!

In [ ]:
#[x / 100.0 for x in np.arange(0, 0.05, 0.01)]


In [ ]:
df['long_slope'].describe()

In [ ]:
df['short_slope'].quantile(quantiles)

In [ ]:
df['actual_slope'].describe()

In [ ]:
df['long_minus_short'].describe()

In [ ]:
entry_slope_threshold       = [x for x in np.arange(0, 0.0005, 0.0001)]
short_entry_slope_threshold = [x for x in np.arange(0, 0.0005, 0.0001)]

for t1 in entry_slope_threshold:
  for t2 in short_entry_slope_threshold:
    entries       = pd.Series(np.where((df['long_slope' ] > t1  ), True, False))
    short_entries = pd.Series(np.where((df['short_slope'] < -t2 ), True, False))

    num_trades = (entries == True).sum() + (short_entries == True).sum()

    if num_trades > 0:    
      pf = vbt.Portfolio.from_signals(
          high              = df['BTCUSDT_High'],
          low               = df['BTCUSDT_Low'],
          open              = df['BTCUSDT_Open'],
          close             = df['BTCUSDT_Close'],
          entries           = entries, # commented out for a short only backtest          
          short_entries     = short_entries,
          td_stop           = prediction_window, # Hold on to the position for 8 bars
          time_delta_format = 'Rows', # Use the row index to calculate the time delta
          # tp_stop = 0.01,
          accumulate        = False,
          # sl_stop = 0.005,
          )    
      
      key = f"entry_slope_threshold_{t1}_short_entry_slope_threshold_{t2}"
      store_backtest_results(key, pf, backtest_result_dict)      

In [ ]:
# Set up entries and exits based on the slope
entry_slope_threshold = 0.0000
short_entry_slope_threshold = 0.0000
entries = pd.Series(np.where((df['long_slope'] > entry_slope_threshold), True, False))
long_minus_short_t = 0.4
entries = pd.Series(np.where((df['long_minus_short'] < long_minus_short_t), True, False))
entries = pd.Series(np.where((df['long_minus_short'] < long_minus_short_t & long_slope > ??), True, False))
#short_entries = pd.Series(np.where((new_df['short_slope'] < -short_entry_slope_threshold), True, False))
#short_entries = pd.Series(np.where((new_df['long_minus_short'] < long_minus_short_t), True, False))
pf = vbt.Portfolio.from_signals(
    high=df['BTCUSDT_High'],
    low=df['BTCUSDT_Low'],
    open=df['BTCUSDT_Open'],
    close=df['BTCUSDT_Close'],
    entries=entries, # commented out for a short only backtest
    # exits = result_df['long_slope']<0.04,
    #short_entries=short_entries,
    td_stop = prediction_window, # Hold on to the position for 8 bars
    time_delta_format = 'Rows', # Use the row index to calculate the time delta
    # tp_stop = 0.01,
    accumulate=False,
    # sl_stop = 0.005,
    )
print(pf.stats())

In [ ]:
# print(f"Num entries = {(entries == True).sum()}")
# print(f"Num short entries = {(short_entries == True).sum()}")
# new_df["long_slope"].describe()

In [ ]:
pf.total_return


In [ ]:
# multiply total_return by 100 and round to 2 decimal places, leaving it as a float
print(f"Total return = {pf.total_return * 100:.2f}%")